In [1]:
import json
#from google.colab import drive

# Get the notebook's filename (usually matches the GitHub repo name)
!ls *.ipynb
notebook_name = "NLPProject.ipynb"  # ← Replace with your filename

# Load and fix the notebook
with open(notebook_name, 'r') as f:
    nb = json.load(f)

# Option A: Remove widgets metadata completely (recommended)
if 'metadata' in nb and 'widgets' in nb['metadata']:
    del nb['metadata']['widgets']

# Option B: Or add the missing state key
# if 'metadata' in nb and 'widgets' in nb['metadata']:
#     nb['metadata']['widgets']['state'] = {}

# Save the fixed version
with open(notebook_name, 'w') as f:
    json.dump(nb, f)

print("Notebook metadata fixed! You can now commit to GitHub.")

NLPProject.ipynb  NLPProject2.ipynb
Notebook metadata fixed! You can now commit to GitHub.


In [2]:
from pprint import pprint
from operator import itemgetter
import nltk
from nltk.corpus import framenet as fn
from nltk.corpus.reader.framenet import PrettyList
nltk.download('framenet_v17')

[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/kierstenwener/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!


True

In [3]:
frame_element_counts = {}
#for each frame, loops through all frame elements
for frame in fn.frames():
    frame_name = frame.name

    for fe_name, fe in frame.FE.items():

        sample_sentences = frame.lexUnit
        num_sentences = len(sample_sentences)

        # Store the count of sentences for each frame element
        if fe_name in frame_element_counts:
            frame_element_counts[fe_name] += num_sentences  # Add the new count to the existing one
        else:
            frame_element_counts[fe_name] = num_sentences

sorted_frame_elements = sorted(frame_element_counts.items(), key=lambda x: x[1], reverse=True)
for fe_name, count in sorted_frame_elements[:10]:
    print(f"Frame Element: {fe_name}, Sample Sentences: {count}")

Frame Element: Time, Sample Sentences: 8170
Frame Element: Manner, Sample Sentences: 7612
Frame Element: Place, Sample Sentences: 7037
Frame Element: Degree, Sample Sentences: 7012
Frame Element: Means, Sample Sentences: 5045
Frame Element: Explanation, Sample Sentences: 4539
Frame Element: Depictive, Sample Sentences: 4091
Frame Element: Purpose, Sample Sentences: 4091
Frame Element: Circumstances, Sample Sentences: 3219
Frame Element: Duration, Sample Sentences: 3120


In [4]:
frames_with_time_ex = {}
for f in fn.frames():
    for x in f.FE:
        if x == "Time":
            frames_with_time_ex[f.name] = f

In [7]:
def align_labels_with_tokens(text, char_labels, offsets):
    token_labels = []
    for (token_start, token_end) in offsets:
        # For special tokens like [CLS] and [SEP], offset is usually (0,0)
        if token_start == token_end:
            token_labels.append("O")
        else:
            # If any character in the token is marked as Time,
            # decide on a label for the entire token.
            token_tag = "O"
            for pos in range(token_start, token_end):
                if pos < len(char_labels) and char_labels[pos] != "O":
                    token_tag = char_labels[pos]
                    break
            token_labels.append(token_tag)
    return token_labels

In [49]:
import torch
from nltk.corpus import framenet as fn
from transformers import BertTokenizerFast

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Map BIO tags to IDs
label2id = {"O": 0, "B-Time": 1, "I-Time": 2}

input_ids_list = []
attention_masks_list = []
labels_list = []

# Find frames that include "Time" as a frame element

for name, frame in frames_with_time_ex.items():
    # Print the frame name for reference
    for lu in frame.lexUnit.values():
        #print(f"\nLexical Unit: {lu['name']}")
        lu_data = fn.lu(lu['ID'])
        for ex in lu_data['exemplars']:
            text = ex['text']
            char_labels = ["O"] * len(text)

            for fe in ex['FE']:
                for i in fe:
                    if i[2] == "Time":
                        start, end = i[0], i[1]
                        if start < end:
                            char_labels[start] = "B-Time"
                            for i in range(start+1, end):
                                char_labels[i] = "I-Time"


            # Tokenize
            tokenized = tokenizer(text, return_offsets_mapping=True, truncation=True, padding="max_length", max_length=128)
            input_ids = tokenized["input_ids"]
            attention_mask = tokenized["attention_mask"]
            offsets = tokenized["offset_mapping"]

            # Map character-level labels to token-level labels
            token_labels = align_labels_with_tokens(text, char_labels, offsets)
            label2id_binary = {"O": 0, "B-Time": 1, "I-Time": 1}  # Map both B-Time and I-Time to 1
            label_ids = [label2id_binary.get(lab, 0) for lab in token_labels]

            # Store tensors
            input_ids_list.append(torch.tensor(input_ids))
            attention_masks_list.append(torch.tensor(attention_mask))
            labels_list.append(torch.tensor(label_ids))

# Final dataset tensors
input_ids_tensor = torch.stack(input_ids_list)
attention_masks_tensor = torch.stack(attention_masks_list)
labels_tensor = torch.stack(labels_list)

print("Tensor shapes:")
print("Input IDs:", input_ids_tensor.shape)
print("Attention Masks:", attention_masks_tensor.shape)
print("Labels:", labels_tensor.shape)

Tensor shapes:
Input IDs: torch.Size([123983, 128])
Attention Masks: torch.Size([123983, 128])
Labels: torch.Size([123983, 128])


In [21]:
indices = (labels_tensor == 1).nonzero(as_tuple=False)
sample_idx, token_idx = indices[0].tolist()
token_id = input_ids_tensor[sample_idx][token_idx]
token = tokenizer.convert_ids_to_tokens([token_id])[0]
print(f"Token: {token}")

Token: when


In [22]:
tokens = tokenizer.convert_ids_to_tokens(input_ids_tensor[sample_idx])
print("Tokens:", tokens)

Tokens: ['[CLS]', 'she', 'had', 'seen', 'no', 'reason', 'to', 'abandon', 'it', 'when', 'she', 'came', 'to', 'med', '##ew', '##ich', 'two', 'years', 'ago', ',', 'even', 'though', 'she', 'might', 'now', 'have', 'been', 'able', 'to', 'afford', 'a', 'car', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [23]:
labels = labels_tensor[sample_idx]
for tok, label in zip(tokens, labels):
    print(f"{tok:15} -> {label.item()}")

[CLS]           -> 0
she             -> 0
had             -> 0
seen            -> 0
no              -> 0
reason          -> 0
to              -> 0
abandon         -> 0
it              -> 0
when            -> 1
she             -> 1
came            -> 1
to              -> 1
med             -> 1
##ew            -> 1
##ich           -> 1
two             -> 1
years           -> 1
ago             -> 1
,               -> 0
even            -> 0
though          -> 0
she             -> 0
might           -> 0
now             -> 0
have            -> 0
been            -> 0
able            -> 0
to              -> 0
afford          -> 0
a               -> 0
car             -> 0
.               -> 0
[SEP]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]           -> 0
[PAD]        

In [50]:
from torch.utils.data import TensorDataset

dataset = TensorDataset(input_ids_tensor, attention_masks_tensor, labels_tensor)


In [51]:
from torch.utils.data import DataLoader, RandomSampler, SubsetRandomSampler
from torch.utils.data import random_split
# Parameters
batch_size = 5
validation_split = 0.5

train_size = int((1 - validation_split) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),  # Shuffle the data
    batch_size=batch_size
)

# Create DataLoader for validation (without shuffling)
val_dataloader = DataLoader(
    val_dataset,
    sampler=SubsetRandomSampler(range(len(val_dataset))),  # Don't shuffle validation data
    batch_size=batch_size
)

In [52]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class FrameElementClassifier(nn.Module):
    def __init__(self, bert_model='bert-base-uncased'):
        super().__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        #self.query_encoder = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
        self.token_projection = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)
        self.classifier = nn.Linear(self.bert.config.hidden_size, 2)
        self.dropout = nn.Dropout(0.1)

    #def forward(self, input_ids, attention_mask, role_ids, role_mask):
    def forward(self, input_ids, attention_mask):
        # Encode sentence
        sentence_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = sentence_outputs.last_hidden_state  # shape: (B, T, H)
        """
        # Encode role label (like "Time" or "Manner")
        role_output = self.bert(input_ids=role_ids, attention_mask=role_mask)
        role_embedding = role_output.last_hidden_state[:, 0, :]  # [CLS] token: shape (B, H)
        role_embedding = self.query_encoder(role_embedding)  # shape: (B, H)
"""
        # Project sentence tokens
        token_embeddings = self.token_projection(token_embeddings)  # shape: (B, T, H)
        logits = self.classifier(token_embeddings)
        # Compute dot product between role embedding and each token
        #role_embedding = role_embedding.unsqueeze(2)  # (B, H, 1)
        #scores = torch.bmm(token_embeddings, role_embedding).squeeze(-1)  # shape: (B, T)

        # Optionally apply attention mask
        #scores = scores.masked_fill(attention_mask == 0, -1e9)
        #probs = torch.softmax(logits, dim=-1)
        #logits = self.classifier(token_embeddings)
        return logits  # Apply softmax for inference or use with CrossEntropyLoss

In [38]:
# Initialize the model
model = FrameElementClassifier()

# Set up device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 17
# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()  # Assuming target_index is 0 for "Non-Time" or 1 for "Time"
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Number of batches to train on
num_batches = 5

# To track accuracy
accuracies = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions_batch = 0
    total_predictions = 0
    print("Target label counts:", torch.bincount(target_index.view(-1)))

    for i, batch in enumerate(train_dataloader):
        if i >= num_batches:
            break
        
        input_ids, attention_mask, target_index = [item.to(device) for item in batch]
        #print("Unique target values:", torch.unique(target_index))
        
        # Zero the gradients before the backward pass
        optimizer.zero_grad()

        # Forward pass
        probs = model(input_ids, attention_mask)

        # Calculate the loss
        loss = loss_fn(probs.view(-1, 2), target_index.view(-1))
        
        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        # Track running loss
        running_loss += loss.item()
        
    with torch.no_grad():
        probs_softmax = torch.softmax(probs, dim=-1)
        predicted_time_token = torch.argmax(time_probs, dim=1)  # (B,)
        #print(f"predicted time token: {predicted_time_token}")

        for b in range(target_index.size(0)):
            time_positions = (target_index[b] == 1).nonzero(as_tuple=True)[0]
            for token_idx in range(input_ids[b].size(0)):
                token_id = input_ids[b][token_idx].item()
                token = tokenizer.convert_ids_to_tokens([token_id])[0]
                true_label = target_index[b][token_idx].item()
                predicted_label = predicted_time_token[b].item()

                #print(f"Token: {token:15} -> True label: {true_label}, Predicted: {predicted_label}")
            for time_pos in time_positions:
                true_token = time_pos.item()  # True token index
                if predicted_time_token[b] == true_token:
                    correct += 1
                total += 1

        batch_accuracy = correct / total if total > 0 else 0
        correct_predictions_batch += correct
        total_predictions += total

    epoch_accuracy = correct_predictions_batch / total_predictions
    print(epoch_accuracy)


Target label counts: tensor([2048])
Token: [CLS]           -> True label: 0, Predicted: 69
Token: brat            -> True label: 0, Predicted: 69
Token: ##by            -> True label: 0, Predicted: 69
Token: was             -> True label: 0, Predicted: 69
Token: intrigued       -> True label: 0, Predicted: 69
Token: by              -> True label: 0, Predicted: 69
Token: mint            -> True label: 0, Predicted: 69
Token: ##on            -> True label: 0, Predicted: 69
Token: and             -> True label: 0, Predicted: 69
Token: soon            -> True label: 1, Predicted: 69
Token: after           -> True label: 1, Predicted: 69
Token: he              -> True label: 1, Predicted: 69
Token: got             -> True label: 1, Predicted: 69
Token: to              -> True label: 1, Predicted: 69
Token: know            -> True label: 1, Predicted: 69
Token: him             -> True label: 1, Predicted: 69
Token: painted         -> True label: 0, Predicted: 69
Token: a               -> Tru

KeyboardInterrupt: 

In [56]:
num_epochs = 10
num_batches = 2
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions_batch = 0
    total_predictions = 0
    
    # Training loop
    for i, batch in enumerate(train_dataloader):
        if i >= num_batches:
            break
        print(i)
        input_ids, attention_mask, target_index = [item.to(device) for item in batch]
        
        optimizer.zero_grad()
        
        probs = model(input_ids, attention_mask)
        loss = loss_fn(probs.view(-1, 2), target_index.view(-1))
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for validation
        for i, batch in enumerate(val_dataloader):
            if i >= num_batches:
                break
            input_ids, attention_mask, target_index = [item.to(device) for item in batch]
            probs = model(input_ids, attention_mask)
            probs_softmax = torch.softmax(probs, dim=-1)
            predicted_tokens = torch.argmax(probs_softmax, dim=-1)

            correct += (predicted_tokens == target_index).sum().item()
            total += target_index.numel()

    accuracy = correct / total
    print(f"Epoch {epoch + 1}, Validation Accuracy: {accuracy:.4f}")



0
1
Epoch 1, Validation Accuracy: 1.0000
0
1
Epoch 2, Validation Accuracy: 0.9945
0
1
Epoch 3, Validation Accuracy: 1.0000
0
1
Epoch 4, Validation Accuracy: 1.0000
0
1
Epoch 5, Validation Accuracy: 1.0000
0
1
Epoch 6, Validation Accuracy: 0.9977
0
1
Epoch 7, Validation Accuracy: 1.0000
0
1
Epoch 8, Validation Accuracy: 0.9992
0
1
Epoch 9, Validation Accuracy: 1.0000
0
1
Epoch 10, Validation Accuracy: 0.9953
